In [ ]:
import numpy as np
import pandas as pd
from plotnine import *
from scipy.io import wavfile

In [ ]:
samplerate, data = wavfile.read('Data/microphone-results.wav')

n = np.arange(len(data))

df = pd.DataFrame({
    'data': data,
    'n':n
})

(ggplot(df, aes('n', 'data'))
 + geom_path())


In [ ]:

def add_fft(df, target_col, samplerate):
    """ Add all the 'fft package' to a dataframe.

    Returns: pandas.DataFrame
    """
    N = len(df.n)
    
    res = pd.DataFrame()
    # Fast Fourier Transform
    res['fft'] = np.fft.fft(df[target_col], N)
    # Power Spectrum Density
    res['PSD'] = res.fft * np.conj(res.fft) / N
    res = res.astype({"PSD": float})
    # Frequency Axis
    res['freq'] = (samplerate/N) * df.n

    return(res)


df_fft = add_fft(df, 'data', samplerate)

def plot_PSD(df):
    # Scale down by 10^9
    df['PSD_A'] = df.PSD/(10**9)
    p = (
        ggplot(df, aes('freq', 'PSD'))
        + geom_path()
        # The human voice frequency ranges from 85Hz to 255Hz.
        + xlim(0, 255)
        + ylab('PSD (10^-9)')
    )

    return(p)

plot_PSD(df_fft)

In [ ]:
def human_voice_only(df):
    df = df.query('freq < 250 & freq >= 85')
    return(df)

df_fft = human_voice_only(df_fft)

In [ ]:
f_filt = np.fft.ifft(df_fft.fft)

wavfile.write('Data/clean_microphone.wav', samplerate, np.int16(f_filt))